The Code Uses Selenium and BeautifulSoup as the main libraries to scrape the data from 99acres. Run all the blocks to get the data from first 30 pages of the website. Page limit can be increased to N. If page Limit will increase then the time taken by the scraper will be more. Time Taken to get the text for 30 pages is 10 Min.

To get the data of properties in four Big Metro Cities in India.
1. Data Collection is done with the help of Web Scraping using Selenium and Language Used is Python.
2. Website We took for Data Collection :- 99acres.com
3. We choose 99 acres because one property is only live for 60 days and we get the new Under Construction or          Ready to move properties easily.
4. Data we scrape consist of 3100 rows and 12 columns.


In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
import time
import requests
import io
import numpy as np
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as ec
from selenium import webdriver
import time
import shutil
import os
import json
from selenium.webdriver.support import expected_conditions as ec
import re
import pandas as pd



/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.3) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [84]:
def get_data_multiple_pages(chrome, city_name, page_sources):
    for i in range(1,30):
        if city_name == 'delhi':
            url = 'https://www.99acres.com/property-in-delhi-ncr-ffid-page-'+str(i)
        elif city_name == 'mumbai':
            url = 'https://www.99acres.com/property-in-mumbai-ffid-page-'+str(i)
        elif city_name == 'chennai':
            url = 'https://www.99acres.com/property-in-chennai-ffid-page-'+str(i)
        elif city_name == 'bangalore':
            url = 'https://www.99acres.com/property-in-bangalore-ffid-page-'+str(i)
        time.sleep(5)
        chrome.execute_script("window.open('');")
        # Switch to the new window and open URL B
        chrome.switch_to.window(chrome.window_handles[1])
        chrome.get(url)
        time.sleep(10)
        new_data = chrome.page_source
        page_sources.append(new_data)
        chrome.close()
        chrome.switch_to.window(chrome.window_handles[0])
    return 



In [5]:
def getBhk(container):
    try:
        bhk = container.find('td', id='srp_tuple_bedroom').text
    except AttributeError:
        bhk = None
        pass
    if bhk is not None:
        try:
            bhk = re.search('(.+)BHK', bhk).group(1)
        except AttributeError:
            bhk = re.search('(.+)RK', bhk).group(1)
    else:
        pass
    return bhk
    

In [6]:
def getBathroom(container):
    try:
        bathroom = container.find('div', id='srp_tuple_bathroom').text
    except AttributeError:
        bathroom = None
    if bathroom is not None:
        bathroom = re.search('(.+)Bath', bathroom).group(1)
    else:
        pass
    return bathroom
    

Helper Functions to get the text out from the Html content.

In [71]:
def getData(house_containers, city_name, address, name, price, price_per_sqft, total_sqft, square_mt_area, bhk, bathroom, availability):
    for container in house_containers:
        addres = container.find_all('h2')[0].text
        print('Address', addres)
        namee = container.find_all('td', class_='list_header_bold srpTuple__spacer10')[0].text
        print('Name', namee)
        pricee = container.find('td', id='srp_tuple_price').text
        print('Price', pricee)
        try:
            pricee = re.search('(.+)₹', pricee).group(1)
        except AttributeError:
            pass
        try:
            price_per_sqft_ = container.find('div', id='srp_tuple_price_per_unit_area').text
        except AttributeError:
            price_per_sqft_ = 0 
        print('price_per_Sqft', price_per_sqft_)
        total_sqft_ = container.find('td', id='srp_tuple_primary_area').text
        total_sqft_ = re.search('(.+)sq.ft.', total_sqft_).group(1)
        square_mt_area_ = container.find('div', id='srp_tuple_secondary_area').text
        square_mt_area_ = re.search('(.+)sq.m.', square_mt_area_).group(1)
        square_mt_area_ = square_mt_area_.lstrip('(') 
        bhk_ = getBhk(container)
        bathroom_ = getBathroom(container)
        availability_ = container.find('div', id='srp_tuple_secondary_tags_0').text
        address.append(addres)
        name.append(namee)
        price.append(pricee)
        price_per_sqft.append(price_per_sqft_)
        total_sqft.append(total_sqft_)
        square_mt_area.append(square_mt_area_)
        bhk.append(bhk_)
        bathroom.append(bathroom_)
        availability.append(availability_)
    return 
        

20


In [82]:
def get_lists(page_sources):
    address = []
    name = []
    price = []
    price_per_sqft = []
    total_sqft = []
    square_mt_area = []
    bhk = []
    bathroom = []
    availability = []
    for i in page_sources:
        html_soup = BeautifulSoup(data, 'html.parser')
        house_containers = html_soup.find_all('table', class_="srpTuple__tupleTable")
        empty_return = getData(house_containers, city_name, address, name, price, price_per_sqft, total_sqft, square_mt_area, bhk, bathroom, availability)
    return address, name, price, price_per_sqft, total_sqft, square_mt_area, bhk, bathroom, availability


In [79]:
def get_dataframe(address, name, price, price_per_sqft, total_sqft, square_mt_area, bhk, bathroom, availability, city_name):
    data = {'Location': address, 'Name_of_The_property': name,'Bhk': bhk, 'Availability': availability, 'Totat_sqft': total_sqft,
        'Price_per_sqft': price_per_sqft, 'Total_price': price, 'Bathroom': bathroom, 'Square_mt_area': square_mt_area, 'City': city_name}
    df = pd.DataFrame(data)
    name = city_name+'_data.csv'
    df.to_csv(name)
    return 

**Main function to be run to get the Scrapper data from 99acres website. The function will ask you to enter the city from which you want to scape data. The script can be used for any city which is listed on 99acres. I have used Selenium and Beautiful soup to get the data from the 99acres and saved that data as a csv file which is starts with the name of city followed by 'data.csv'**.

In [1]:
if __name__ == "__main__":
    chrome = webdriver.Chrome(executable_path=r'/home/katik/Downloads/chromedriver')
    chrome.get("https://www.99acres.com/")
    print(chrome.title)
    textField = "keyword"
    wait = WebDriverWait(chrome, 20)
    properties = wait.until(lambda driver: driver.find_element_by_xpath(
    '//*[@id="keyword"]'))
    city_name = input("Enter the city for which you want to search :-")
#     city_name = 'delhi'
    properties.send_keys(city_name)
    wait.until(ec.visibility_of_element_located((By.XPATH, '//*[@id="submit_query"]')))
    chrome.find_element_by_xpath('//*[@id="submit_query"]').click()
    time.sleep(10)
    page_sources = []
    data = chrome.page_source
    page_sources.append(data)
    get_data_multiple_pages(chrome, city_name, page_sources)
    chrome.close()
    address, name, price, price_per_sqft, total_sqft, square_mt_area, bhk, bathroom, availability = get_lists(page_sources)
    get_dataframe(address, name, price, price_per_sqft, total_sqft, square_mt_area, bhk, bathroom, availability, city_name)
    chrome.quit()

In [90]:
dataframe_bangalore = pd.read_csv('bangalore_data.csv')
dataframe_delhi = pd.read_csv('delhi_data.csv')
dataframe_mumbai = pd.read_csv('mumbai_data.csv')
dataframe_chennai = pd.read_csv('chennai_data.csv')

In [91]:
import pandas as pd
df = pd.concat([dataframe_bangalore, dataframe_chennai, dataframe_delhi, dataframe_mumbai], ignore_index=True)

In [92]:
df.to_csv('MetroCitiesDataNew.csv')